In [1]:
import numpy as np
import pandas as pd

In [18]:
data_path="C://Users//Dimitri//Desktop//ENSAE3A//Apprentissage en ligne//OnlinePortfolio//TreatedData//DataNyse.csv"
# Frequence of rebalancing the portfolio
ndays_rebal = 20

In [13]:
# Load the data
data_nyse = pd.read_csv(data_path, index_col="Date")
# Convert data index to datetime
data_nyse.index = pd.to_datetime(data_nyse.index)
# Print the data for a quick check
print(data_nyse)

               nwl     fhn       cmi     cma     wfc     cms      ups     nem  \
Date                                                                            
2001-09-21  17.690  29.537    6.4215  43.127  16.944  15.085   40.068  19.856   
2001-09-24  18.018  29.955    6.8668  44.278  17.225  15.085   40.161  19.260   
2001-09-25  18.181  30.751    6.9695  45.840  17.372  14.887   41.359  19.565   
2001-09-26  18.392  31.850    6.7469  45.112  17.423  14.494   39.861  20.832   
2001-09-27  18.852  32.147    6.8838  45.967  17.431  14.287   40.083  21.083   
2001-09-28  18.945  32.113    7.0640  46.753  17.735  14.368   41.900  21.039   
2001-10-01  19.075  31.814    6.8754  46.376  17.735  14.368   40.826  22.134   
2001-10-02  19.347  32.331    7.1667  45.744  17.796  14.454   40.387  21.891   
2001-10-03  19.895  31.859    7.2006  45.608  17.859  14.793   41.195  20.581   
2001-10-04  19.903  30.760    7.2605  45.325  17.654  15.085   41.917  20.562   
2001-10-05  20.112  29.935  

In [67]:
# Get the returns for every ndays_rebal and select only every ndays_rebal-th date
data_returns = data_nyse.pct_change(ndays_rebal).ix[::ndays_rebal, :]
data_returns = data_returns.shift(-1)

In [68]:
def exp_weights(pvec, xvec, eta):
    norm_coef = (pvec * xvec).sum()
    wvec = pd.Series(index=pvec.index)
    for i in pvec.index:
        wvec[i] = pvec[i] * np.exp (eta * xvec[i] / norm_coef)
    return wvec / wvec.sum()

In [69]:
# Test for exp_weights
x = 100 * data_returns.iloc[1, 0:10]
p = pd.Series(index=x.index, data=1/x.shape[0])
w = exp_weights(p, x, 0.5)
print(w)

nwl     0.108676
fhn     0.151256
cmi     0.179955
cma     0.124699
wfc     0.113739
cms     0.102684
ups     0.094980
nem     0.022668
lmt     0.030689
tgna    0.070654
dtype: float64


In [75]:
def weights_eg(returns, eta, pinit):
    w = pd.DataFrame(index=returns.index, columns=returns.columns)
    for t in range(0, w.shape[0]):
        if t==0:
            w.iloc[t, :] = pinit
        else :
            coefs = exp_weights(w.iloc[t-1, :], returns.iloc[t-1, :], eta)
            w.iloc[t, :] = coefs
    return w

In [78]:
# Test for weights_eg
x = 100 * data_returns.iloc[:, 0:10]
pinit = pd.Series(index=x.columns, data=1/len(x.columns))
w = weights_eg(x, 0.01, pinit)

In [79]:
w

,nwl,fhn,cmi,cma,wfc,cms,ups,nem,lmt,tgna
Date,,,,,,,,,,
2001-09-21,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2001-10-19,0.10686,0.0965511,0.100652,0.0939254,0.0954966,0.0994181,0.0998341,0.0982424,0.104628,0.104391
2001-11-16,0.107395,0.0976871,0.102196,0.0946591,0.0960633,0.0998009,0.10006,0.0956458,0.102491,0.104002
2001-12-17,0.107283,0.0968551,0.104044,0.0973646,0.0957099,0.0990394,0.0999939,0.0942245,0.103328,0.102157
2002-01-16,0.105358,0.0940996,0.0987963,0.0983809,0.097756,0.100815,0.0996726,0.0974531,0.104581,0.103087
2002-02-14,0.105221,0.0933458,0.099727,0.0978953,0.0973635,0.0989215,0.0989822,0.0995244,0.105519,0.1035
2002-03-15,0.106377,0.0927787,0.101362,0.098645,0.0974465,0.0979384,0.0987931,0.0979507,0.105588,0.103121
2002-04-15,0.103873,0.0936242,0.101747,0.0960355,0.0955762,0.0946708,0.0977717,0.107182,0.106633,0.102887
2002-05-13,0.095555,0.0897716,0.112922,0.0930045,0.091928,0.110001,0.0981113,0.101581,0.103097,0.104028
